In [1]:

from PIL import Image 
from keras.applications.mobilenet import preprocess_input
import base64 
from io import BytesIO 
import json 
import random 
import cv2 
from keras.models import load_model 
import numpy as np 
threshold = 0.70  
font = cv2.FONT_HERSHEY_SIMPLEX
from keras.preprocessing import image 


model = load_model('Emotion.h5',compile=False)
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml') 
#haarcascade_frontalface_default/haarcascade_frontalface_alt_tree



Using Theano backend.
C:\Users\Younes\Anaconda3\envs\py\lib\site-packages\keras\utils\conv_utils.py:82: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.copy(kernel[slices])


In [2]:
def face_extractor(img):
    faces = face_cascade.detectMultiScale(img, 1.3, 5) 
    if faces is (): 
        return None 

    for (x,y,w,h) in faces: 
        #cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        cropped_face = img[y:y+h, x:x+w] 
     
    return cropped_face 

In [6]:
#class = ['Angry','Disgust','Fear','Happy','Neutral','Sad','Surprise']


video_capture = cv2.VideoCapture(0) 
while True: 
    _, frame = video_capture.read() 
   # frameg=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    face=face_extractor(frame) 
    if type(face) is np.ndarray: 
        face = cv2.resize(face, (224, 224)) 
        im = Image.fromarray(face,'RGB') 
        img_array = np.array(im) 
        img_array_expanded_dims = np.expand_dims(img_array, axis=0) 
        img_array = preprocess_input(img_array_expanded_dims)
       
        pred = model.predict(img_array)
        prediction = np.argmax(pred[0]) 
        #label=class_labels[pred.argmax()]
        
        
        probabilityValue =np.amax(pred)
        if probabilityValue > threshold:
            
            if prediction==0:
                name='Angry :'
            elif prediction==3:
                name='Happy :'
            elif prediction==4:
                name='Neutral :'
            elif prediction==6:
                name='Surprise :'    
            #cv2.putText(frame,name, (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2) 
            #cv2.putText(frame, name+str(round(probabilityValue*100,2) )+"%", (50, 75), font, 0.75, (0, 0, 255), 2, cv2.LINE_AA)
            text = "{}: {:.2f}%".format(name, probabilityValue * 100)
            faces = face_cascade.detectMultiScale(frame, 1.3, 5) 
            for (x,y,w,h) in faces: 
                cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
                #cropped_face = img[y:y+h, x:x+w] 
            
                cv2.putText(frame, text, ((x,y-5)),
                cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,152, 169), 2)
    else: 
        cv2.putText(frame,'No Face Found',(20,60),cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3) 
    cv2.imshow('Video', frame) 
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break 
video_capture.release() 
cv2.destroyAllWindows() 